In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

import requests
from time import sleep
from bs4 import BeautifulSoup
import csv

import secrets

In [2]:
valid_urls = set()

In [3]:
# response = requests.get('http://duckduckgo.com/html/?q=go&ia=web')
# soup = BeautifulSoup(response.text,"lxml")
# # soup.prettify()
# for a in soup.find_all('a'):
#     href = a.get('href')
#     if href:# and href[0:4] == 'http':
#         href = href.split('?')[0]
#         valid_urls.add(href)
# valid_urls

In [35]:
def get_rows_for_search_term(q, mkt='en-US', count=10):
    search_url = "https://api.cognitive.microsoft.com/bing/v7.0/search"
    headers = {"Ocp-Apim-Subscription-Key": secrets.azure_api_key}
    params  = {"q": q, "mkt": mkt, "count": count}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    return [[q, mkt, v['url']] for v in search_results['webPages']['value']]

In [31]:
csv_rows = [['search_term','market','url']]

In [15]:
used_words = set()

In [30]:
# For some reason, the Bing API is not allowing non-english speaking markets.
# Perhaps a restriction for trial accounts.
english_speaking_markets = [
  'en-AU',
  'en-CA',
  'en-IN',
  'en-ID',
  'en-MY',
  'en-NZ',
  'en-PH',
  'en-ZA',
  'en-GB',
  'en-US'
]
esm_len = len(english_speaking_markets)
def get_random_mkt():
    return english_speaking_markets[np.random.randint(esm_len)]
max_results = 50

In [40]:
with open('data/random_words.txt', 'r') as random_words:
    for q in random_words:
        q = q.strip()
        if q in used_words: continue
        sleep(0.34)
        mkt = get_random_mkt()
        csv_rows.extend(get_rows_for_search_term(q, mkt, max_results))
        used_words.add(q)
        if len(csv_rows) > 1e4: break

In [41]:
len(csv_rows)

10001

In [42]:
len(used_words)

210

In [43]:
with open('data/bing_search_results.csv', 'w') as results_csv:
    writer = csv.writer(results_csv)
    writer.writerows(csv_rows)